In [44]:
node=4
ppn=40
solutions = ['none', 'intent']
apps = ['sync-write-1d-contig-interleaved', 'sync-write-1d-contig-strided', 'sync-write-1d-contig-contig']

In [45]:
!ls cmake_test.sh_1_h5bench_sync_none_intent_1_40_sync-write-1d-contig*

cmake_test.sh_1_h5bench_sync_none_intent_1_40_sync-write-1d-contig-contig-read-full.out
cmake_test.sh_1_h5bench_sync_none_intent_1_40_sync-write-1d-contig-contig-read-partial.out
cmake_test.sh_1_h5bench_sync_none_intent_1_40_sync-write-1d-contig-contig-read-strided.out
cmake_test.sh_1_h5bench_sync_none_intent_1_40_sync-write-1d-contig-contig.out
cmake_test.sh_1_h5bench_sync_none_intent_1_40_sync-write-1d-contig-interleaved.out
cmake_test.sh_1_h5bench_sync_none_intent_1_40_sync-write-1d-contig-strided.out


In [46]:
import re
values = {}
for app in apps:
    values[app] = {}
    for solution in solutions:
        values[app][solution] = {}
        file_path=f"cmake_test.sh_{node}_h5bench_sync_none_{solution}_{node}_{ppn}_{app}.out"
        with open(file_path, 'r') as fp:
            lines = fp.readlines()
            for line in lines:
                # check if string present on a current line
                if line.find("SUCCESS") != -1:
                    log_dir = re.search('at (.+?)\)', line).group(1)
                    log_file = f"{log_dir}/stdout"
                    values[app][solution]["log"] = log_file
                    values[app][solution]["size"] = {}
                    values[app][solution]["time"] = {}
                    values[app][solution]["rate"] = {}
                    with open(log_file, 'r') as log_fp:
                        log_lines = log_fp.readlines()
                        for log_ln in log_lines:
                            # check if string present on a current line
                            if log_ln.find("ranks:") != -1:
                                #print(log_ln)
                                ranks = re.search('ranks: (.+?)', log_ln).group(1)
                                values[app][solution]["ranks"] = ranks
                            elif log_ln.find("size:") != -1:
                                op = re.search('Total (.+?) size:', log_ln).group(1)
                                size = re.search('size: (.+?) GB', log_ln).group(1)
                                values[app][solution]["size"][op] = size
                            elif log_ln.find("time:") != -1:
                                #print(log_ln)
                                op = re.search('(.+?) time:', log_ln).group(1)
                                time = re.search('time: (.+?) s', log_ln).group(1)
                                values[app][solution]["time"][op] = time
                            elif log_ln.find("rate:") != -1:
                                op = re.search(' (.+?) rate:', log_ln).group(1)
                                rate = re.search('rate: (.+?)', log_ln).group(1)
                                values[app][solution]["rate"][op] = rate
                    


In [47]:
for app, app_val in values.items():
    for solution, solution_val in app_val.items():
        print(app, solution, solution_val['rate'] if 'rate' in solution_val else solution_val)

sync-write-1d-contig-interleaved none {'Raw write': '2', 'Observed write': '2'}
sync-write-1d-contig-interleaved intent {'Raw write': '2', 'Observed write': '2'}
sync-write-1d-contig-strided none {'Raw write': '1', 'Observed write': '1'}
sync-write-1d-contig-strided intent {'Raw write': '1', 'Observed write': '1'}
sync-write-1d-contig-contig none {'Raw write': '6', 'Observed write': '6'}
sync-write-1d-contig-contig intent {'Raw write': '6', 'Observed write': '5'}
